In [3]:
from torch.nn import Conv2d, MaxPool2d, Dropout, Linear, ReLU, CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import torchvision
import pathlib
import pickle
import torch
import os
import gc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!unzip drive/MyDrive/hackathon_data.zip

In [ ]:
import pathlib

images_path = 'bev_classification/images'
test_path = 'bev_classification/images/test'
train_path = 'bev_classification/images/train'

if not pathlib.Path(test_path).exists():
    pathlib.Path(test_path).mkdir()

if not pathlib.Path(train_path).exists():
    pathlib.Path(train_path).mkdir()

for file in pathlib.Path(images_path).iterdir():
    if file.name == '.DS_Store' or \
        file == test_path or \
        file == train_path:
        continue

    if 'test' in file.name:
        for data in pathlib.Path(file).iterdir():
            # Only iterate over the class directories
            if 'image-datasets' == data.name or 'input' in data.name or data.name == '.DS_Store': continue

            if pathlib.Path(f'{test_path}/{data.name}').exists():
                # print(data.name)
                for f in pathlib.Path(data).iterdir():
                    pathlib.Path(f).rename(f'{test_path}/{data.name}/{f.name}')

            else:
                pathlib.Path(data).rename(f'{test_path}/{data.name}')
            # print(f'{test_path}/{data.name}')
    else:
        for data in pathlib.Path(file).iterdir():
            # Only iterate over the class directories
            if 'image-datasets' == data.name or 'input' in data.name or data.name == '.DS_Store': continue

            if pathlib.Path(f'{train_path}/{data.name}').exists():

                for f in pathlib.Path(data).iterdir():
                    pathlib.Path(f).rename(f'{train_path}/{data.name}/{f.name}')

            else:
                pathlib.Path(data).rename(f'{train_path}/{data.name}')

In [ ]:
import pathlib

train_path = 'bev_classification/images/train'
val_path = 'bev_classification/images/val'

if not pathlib.Path(val_path).exists():
    pathlib.Path(val_path).mkdir()

for id in pathlib.Path(train_path).iterdir():
    if not pathlib.Path(f"{val_path}/{id.name}").exists():
        pathlib.Path(f"{val_path}/{id.name}").mkdir()

    files = [file for file in pathlib.Path(id).iterdir()]
    val_files = files[:len(files)//8]

    [pathlib.Path(f).rename(f'{val_path}/{id.name}/{f.name}') for f in val_files]


In [ ]:
class BevDataset(Dataset):
  def __init__(self, root, size=224, split='train'):
    self.split = split
    postfix = split
    root = os.path.join(root, 'bev_classification', 'images')
    self.dataset_folder = torchvision.datasets.ImageFolder(os.path.join(root, postfix) ,transform = transforms.Compose([transforms.Resize((size,size)),transforms.ToTensor()]))

  def __getitem__(self,index):
    img = self.dataset_folder[index]
    path = self.dataset_folder.imgs[index]
    return img[0], img[1], path[0]

  def __len__(self):
    return len(self.dataset_folder)

In [ ]:
import torchvision.models as models

class VGGIntermediate(nn.Module):
    def __init__(self):
        super(VGGIntermediate, self).__init__()
        self.vgg = models.vgg19(pretrained=True)
        self.set_up_vgg()

    def set_up_vgg(self):
        for param in self.vgg.parameters():
            param.requires_grad = False

        num_features = self.vgg.classifier[-1].in_features  # Get the number of input features for the final layer
        self.vgg.classifier[-1] = Linear(num_features, 99)
        # Optionally, you may want to initialize the new layer weights
        # Initialize weights with Xavier initialization
        torch.nn.init.xavier_uniform_(self.vgg.classifier[-1].weight)
        # Optionally, initialize biases to zeros
        torch.nn.init.zeros_(self.vgg.classifier[-1].bias)

    def forward(self, X):
        return self.vgg(X)

In [ ]:
# Hyper parameters
# With batch_size 50, there will be 1776 iterations over the dataset per epoch
batch_size = 20
num_epochs = 5
lr = 1e-4

epoch: 1 batch: 2025 accuracy: 22.64% val accuracy 39.20% loss: 3.2424 val loss: 2.3560:   4%|▍         | 2026/48580 [05:31<1:10:22, 11.03it/s]

In [ ]:
train_losses = [0]
train_accuracy = [0]

val_losses = [0]
val_accuracy = [0]

model_path = 'model/cuda-model.pkl'

In [ ]:
def fine_tune():
    try:
        gc.collect()
        device = torch.device('cuda')
        if not pathlib.Path(model_path).exists():
            model = VGGIntermediate().to(device)
        else:
            print('Model Found!')
            print('Loading model...')
            with open(model_path, 'rb') as f:
                model = pickle.load(f).to(device)

        print()
        train_loader = DataLoader(BevDataset('.'), batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(BevDataset('.', split='val'), batch_size=batch_size, shuffle=True)

        # Only have 10 validation checks per epoch
        val_check = len(train_loader) // 5

        criterion = CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        loop = tqdm(total=len(train_loader)*num_epochs, position=0)

        for epoch in range(num_epochs):
            train_step_losses = []
            train_step_accuracy = []
            for batch, (x, y_truth, _) in enumerate(train_loader):
                x, y_truth = x.to(device), y_truth.to(device)

                optimizer.zero_grad()

                y_hat = model(x)

                accuracy = (y_hat.argmax(1) == y_truth).float().mean()
                train_step_accuracy.append(accuracy.item())

                loss = criterion(y_hat, y_truth)
                train_step_losses.append(loss.item())

                loss.backward()
                optimizer.step()

                if (batch + 1) % val_check == 0:
                    print('Validation check')
                    val_loop = tqdm(total=len(val_loader), position=0)

                    val_batch_loss = []
                    val_batch_accuracy = []
                    for batch, (x, y_truth, _) in enumerate(val_loader):
                        x, y_truth = x.to(device), y_truth.to(device)

                        optimizer.zero_grad()

                        y_hat = model(x)

                        accuracy = (F.softmax(y_hat,1).argmax(1) == y_truth).float().mean()
                        val_batch_accuracy.append(accuracy.item())

                        loss = criterion(y_hat, y_truth)
                        val_batch_loss.append(loss.item())

                        loss.backward()
                        optimizer.step()
                        val_loop.update(1)
                        val_loop.set_description(f'val batch: {batch} val accuracy: {accuracy*100:.2f}% val loss: {loss:.4f}')

                    val_losses.append(sum(val_batch_loss) / len(val_batch_loss))
                    val_accuracy.append(sum(val_batch_accuracy) / len(val_batch_accuracy))

                    train_losses.append(sum(train_step_losses) / len(train_step_losses))
                    train_accuracy.append(sum(train_step_accuracy) / len(train_step_accuracy))

                loop.update(1)
                loop.set_description(f'epoch: {epoch+1} batch: {batch} accuracy: {train_accuracy[-1]*100:.2f}% val accuracy {val_accuracy[-1]*100:.2f}% loss: {train_losses[-1]:.4f} val loss: {val_losses[-1]:.4f}')

            print('Saving model...')
            with open(model_path, 'wb') as f:
                pickle.dump(model, f)
            print('Model saved.')
    except KeyboardInterrupt:
        print('Saving model...')
        with open(model_path, 'wb') as f:
            pickle.dump(model, f)
        print('Model saved.')

# fine_tune()

Model Found!
Loading model...



epoch: 5 batch: 3886 accuracy: 64.60% val accuracy 71.68% loss: 1.2531 val loss: 1.0059: 100%|██████████| 19435/19435 [2:12:10<00:00,  2.45it/s]
epoch: 1 batch: 775 accuracy: 64.60% val accuracy 71.68% loss: 1.2531 val loss: 1.0059:   4%|▍         | 776/19435 [02:47<1:04:23,  4.83it/s]

Validation check


val batch: 553 val accuracy: 100.00% val loss: 0.2774: 100%|██████████| 554/554 [15:34<00:00,  1.69s/it]
epoch: 1 batch: 1552 accuracy: 65.42% val accuracy 72.22% loss: 1.2173 val loss: 0.9801:   8%|▊         | 1553/19435 [07:36<1:01:56,  4.81it/s]

Validation check


...

val batch: 553 val accuracy: 100.00% val loss: 0.5377: 100%|██████████| 554/554 [04:49<00:00,  1.91it/s]
epoch: 2 batch: 2329 accuracy: 66.13% val accuracy 72.99% loss: 1.1884 val loss: 0.9416:  32%|███▏      | 6217/19435 [36:32<49:51,  4.42it/s]

Validation check


val batch: 553 val accuracy: 100.00% val loss: 0.1898: 100%|██████████| 554/554 [04:48<00:00,  1.92it/s]
epoch: 2 batch: 3106 accuracy: 65.74% val accuracy 72.93% loss: 1.2002 val loss: 0.9352:  36%|███▌      | 6994/19435 [41:18<43:41,  4.75it/s]

Validation check


val batch: 553 val accuracy: 100.00% val loss: 0.0033: 100%|██████████| 554/554 [04:46<00:00,  1.93it/s]
epoch: 2 batch: 3856 accuracy: 65.64% val accuracy 73.49% loss: 1.2013 val loss: 0.9265:  40%|███▉      | 7744/19435 [45:58<44:29,  4.38it/s]

In [ ]:
gc.collect()

device = torch.device('mps')
print('Model Found!')
print('Loading model...')
with open(model_path, 'rb') as f:
    model = pickle.load(f).to(device)
test_dataset = BevDataset('.', split='test')
test_loader = DataLoader(BevDataset('.', split='test'), batch_size=batch_size, shuffle=True)
test_classes = test_dataset.dataset_folder.classes

In [ ]:
def write_predictions(y_hat, paths, f):
    class_nums = F.softmax(y_hat,1).argmax(1)
    class_labels = [test_classes[c] for c in class_nums]
    for path, label in zip(paths, class_labels):
        f.write(f'{path},{label}\n')

In [ ]:
def write_five_predictions(y_hat, paths, f):
    for i in range(y_hat.size()[0]):
        class_nums = torch.topk(y_hat[i,:].flatten(), 5).indices
        class_labels = [str(test_classes[c]) for c in class_nums]
        f.write(f'{paths[i]},{",".join(class_labels)}\n')

In [ ]:
with open('./drive/MyDrive/test_edited.txt', 'w') as f, open('./drive/MyDrive/test_edited_five.txt', 'w') as f_f:
    loop = tqdm(total=len(test_loader), position=0)
    for x, _, path in test_loader:
        y_hat = model(x.to(device))
        write_predictions(y_hat, path, f)
        write_five_predictions(y_hat, path, f_f)
        loop.update(1)